In [1]:
# インポート
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
import tensorflow.keras.layers as layers
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Input
import numpy as np

In [2]:
# カスタムレイヤの構築　https://hironsan.hatenablog.com/entry/how-to-create-keras-custom-layer
disp_flg=True
class CustomDense(layers.Layer):
    def __init__(self, units=10):
        super(CustomDense, self).__init__()
        self.units = units
        # print(units)

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(1,self.units), initializer="random_normal", trainable=True
        )
        self.c = tf.constant([0,1,2,3],shape=[1,4], dtype="float32")

    def call(self, x):
        # return tf.matmul(x, self.c,  transpose_b=True)
        return tf.matmul(x, self.b,  transpose_a=True)



if disp_flg==True:
    inputs = keras.Input(shape=(1,4))
    outputs = CustomDense(4)(inputs)
    model = keras.Model(inputs, outputs)
    model.summary()

    a = tf.constant([0,1,2,3],shape=[1,4], dtype="float32")
    out = model(a)
    print(out)

    # 以下と同じ
    out = tf.matmul(a,a,transpose_a=True)
    print(out)
    

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 4)]            0         
_________________________________________________________________
custom_dense (CustomDense)   (None, 4, 4)              20        
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________
tf.Tensor(
[[ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 1.8973224e-02 -3.5656266e-02 -1.1627777e-01 -1.3140464e-04]
 [ 3.7946448e-02 -7.1312532e-02 -2.3255554e-01 -2.6280928e-04]
 [ 5.6919672e-02 -1.0696880e-01 -3.4883332e-01 -3.9421392e-04]], shape=(4, 4), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0.]
 [0. 1. 2. 3.]
 [0. 2. 4. 6.]
 [0. 3. 6. 9.]], shape=(4, 4), dtype=float32)


In [4]:
# モデルの構築
mode=1
# 1. Sequencial Model

# 2. Functional API (そのまま使う)
if mode==1:
    # img_inputs = keras.Input(shape=(32, 32, 3))
    inputs = keras.Input(shape=(784,))
    x = layers.Dense(64, activation="relu")(inputs)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(10)(x)
    model = keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")

if mode==2:
    inputs = keras.Input(shape=(4,))
    outputs = CustomDense(4)(inputs)
    model = keras.Model(inputs, outputs)



In [5]:
# 他入力他出力モデルの作成

In [6]:
# モデルのトレーニング

mode=0
# データのロード
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

if mode==0: # 通常の方法
    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=keras.optimizers.RMSprop(),
        metrics=["accuracy"],
    )

    history = model.fit(x_train, y_train, batch_size=64, epochs=2, validation_split=0.2)

    test_scores = model.evaluate(x_test, y_test, verbose=2)
    print("Test loss:", test_scores[0])
    print("Test accuracy:", test_scores[1])




Epoch 1/2
750/750 [==============================] - 3s 4ms/step - loss: 0.3468 - accuracy: 0.9021 - val_loss: 0.1941 - val_accuracy: 0.9427
Epoch 2/2
750/750 [==============================] - 2s 3ms/step - loss: 0.1603 - accuracy: 0.9524 - val_loss: 0.1498 - val_accuracy: 0.9538
313/313 - 0s - loss: 0.1464 - accuracy: 0.9565
Test loss: 0.14641624689102173
Test accuracy: 0.9564999938011169


In [34]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
import tensorflow.keras.layers as layers
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Input
import numpy as np
from tensorflow.keras import activations
# データセットのロード
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0#0~1の値で正規化

# モデルの構築
model_sq = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model_sq.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

# 学習
# model_sq.fit(x_train, y_train, epochs=10)
# モデル性能の評価
# model_sq.evaluate(x_test,  y_test, verbose=2)


# カスタムモデルで作成

# ・カスタムレイヤ
# Dense
class MyDense(layers.Layer):
    def __init__(self, units=10, activation=None):
        super(MyDense, self).__init__()
        self.units = units
        self.activation_name = activation

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="ones",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="zeros", trainable=True
        )

        if self.activation_name==None:
            self.activation=None
        elif self.activation_name=="relu":
            self.activation = activations.relu
        # self.c = tf.constant([0,1,2,3],shape=[1,4], dtype="float32")

    def call(self, inputs):
        out = tf.matmul(inputs, self.w) + self.b
        if self.activation is not None:
            out = self.activation(out)

        return out


In [38]:
x = x_test[0].reshape(1,784)
# x = np.zeros([1,784])
# print(x)

inputs = keras.Input(shape=(784,))
outputs = MyDense(128,'relu')(inputs)
model = keras.Model(inputs, outputs)
model.summary()

out = model(x)
print(out)

Model: "functional_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_26 (InputLayer)        [(None, 784)]             0         
_________________________________________________________________
my_dense_22 (MyDense)        (None, 128)               100480    
Total params: 100,480
Trainable params: 100,480
Non-trainable params: 0
_________________________________________________________________
tf.Tensor(
[[72.36864 72.36864 72.36864 72.36864 72.36864 72.36864 72.36864 72.36864
  72.36864 72.36864 72.36864 72.36864 72.36864 72.36864 72.36864 72.36864
  72.36864 72.36864 72.36864 72.36864 72.36864 72.36864 72.36864 72.36864
  72.36864 72.36864 72.36864 72.36864 72.36864 72.36864 72.36864 72.36864
  72.36864 72.36864 72.36864 72.36864 72.36864 72.36864 72.36864 72.36864
  72.36864 72.36864 72.36864 72.36864 72.36864 72.36864 72.36864 72.36864
  72.36864 72.36864 72.36864 72.36864 72.36864 72.3686

In [8]:
# 比較
x = x_test[0].reshape(1,784)
inputs = keras.Input(shape=(784,))
outputs = Dense(128)(inputs)
print(out.shape)
model.summary()

(1, 128)
Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
my_dense_4 (MyDense)         (None, 128)               100480    
Total params: 100,480
Trainable params: 100,480
Non-trainable params: 0
_________________________________________________________________
